# Train Model

In [57]:
import sys
sys.path.append("..")
%reload_ext autoreload
%autoreload 2

In [58]:
import pandas as pd
from sklearn_pandas import cross_val_score, DataFrameMapper
from pandas import Categorical
from sklearn.ensemble import RandomForestClassifier
import mwapi

In [59]:
from wikidit.models import featurize, load_wp10, OrdinalClassifier

In [60]:
input_file = "../data/enwiki.labeling_revisions.w_features.nettrom_30k.csv.gz"
revisions = load_wp10(input_file)

# Create a pipeline

In [21]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, PolynomialFeatures

In [22]:
import xgboost as xgb

import dill

In [33]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.base import BaseEstimator, TransformerMixin

sqrt_cols = ['words',
             
             'headings',
             'sub_headings',
             'images',
             'categories',
             'wikilinks',

             'who_templates',
             'main_templates',
             'cite_templates',
             # infobox as a binary
             'citation_needed',
             'other_templates',

             'ref',
             'smartlists',
             'coordinates']

binarized_cols = ['coordinates', 'infoboxes']

mapper = DataFrameMapper([
    (sqrt_cols, FunctionTransformer(func=np.sqrt)),
    (binarized_cols, FunctionTransformer(func=lambda x: x.astype(bool)))
])

clf = xgb.XGBClassifier(max_depth=5,
                        learning_rate=0.01, 
                        n_estimators=100, silent=True, 
                        objective='binary:logistic')

In [85]:
dtrain = xgb.DMatrix(mapper.fit_transform(revisions))
param = {'max_depth':10, 'eta':1, 'silent':1, 'objective':'binary:logistic'}
num_round = 10


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [92]:
dtrain = xgb.DMatrix(mapper.fit_transform(revisions), label=revisions['wp10'] > 'Start')

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [93]:
xgb.cv(param, dtrain, enum_round, nfold=5)

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.073379,0.001198,0.110258,0.003556
1,0.064119,0.001073,0.109394,0.001530
2,0.059570,0.001968,0.107235,0.002285
3,0.056486,0.001921,0.108284,0.002771
4,0.053225,0.001978,0.108037,0.001737
5,0.050472,0.001933,0.108253,0.001935
6,0.048128,0.001854,0.108870,0.001781
7,0.046077,0.001280,0.110196,0.001407
8,0.044419,0.001446,0.110196,0.001279
9,0.040263,0.003480,0.110165,0.001323


Get categories

In [39]:

def _parallel_fit_estimator(estimator, X, y, cat):
    """Private function used to fit an estimator to a class within a job."""
    touse = (y >= cat)
    y_transformed = y[touse] > cat
    estimator.fit(X[touse, :], y_transformed)
    return estimator


class OrdinalClassifier(_BaseComposition, ClassifierMixin, TransformerMixin):

    def __init__(self, estimator, n_jobs=None, proba_transform=None, left=True):
        self.estimator = estimator
        self.n_jobs = n_jobs
        self.proba_transform = proba_transform
        self.left = left

    @property
    def named_estimators(self):
        return Bunch(**dict(self.estimator))

    def fit(self, X, y, categories='auto'):
        if not (isinstance(y, pd.Series) and hasattr(y, "cat")):
            raise ValueError("y must be pd.Series object with dtype Categorical")

        # this is hard-coded for categorical variables
        self.classes_ = y.cat.categories

        categories = self.classes_[:-1]
    
        # order of estimators
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
                delayed(_parallel_fit_estimator)(clone(self.estimator), X, y, cat)
                for cat in categories)

        self.named_estimators_ = Bunch(**dict())
        for k, e in zip(self.classes_[:-1], self.estimators_):
            self.named_estimators_[k] = e
        return self

    def predict(self, X):
        out = np.argmax(self.predict_proba(X), axis=1)
        out = pd.Categorical.from_codes(out, categories=self.classes_, ordered=True)
        return out

    def _collect_log_probas(self, X):
        """Collect results from clf.predict calls. """
        if hasattr(clf, "predict_log_proba"):
            return [clf.predict_log_proba(X) for clf in self.estimators_]
        else:
            return [np.log(clf.predict_proba(X)) for clf in self.estimators_]

    def _predict_log_proba(self, X):
        """Predict log class probabilities for X"""
        out = np.empty((X.shape[0], len(self.classes_)))
        for i, logp in enumerate(self._collect_log_probas(X)):
            if i > 0:
                # add log conditional probability
                logp += out[:, (i, )]
            out[:, i:(i + 2)] = logp
        return out

    @property
    def predict_log_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_log_proba
    
    def predict_proba(self, X):
        return np.exp(self.predict_log_proba(X))

    def transform(self, X):
        if self.proba_transform:
            return self.predict_proba(X)
        else:
            return self.predict(X)
    
    def set_params(self, **params):
        """ Setting the parameters for the voting classifier
        Valid parameter keys can be listed with get_params().
        Parameters
        ----------
        **params : keyword arguments
            Specific parameters using e.g. set_params(parameter_name=new_value)
            In addition, to setting the parameters of the ``VotingClassifier``,
            the individual classifiers of the ``VotingClassifier`` can also be
            set or replaced by setting them to None.
        Examples
        --------
        # In this example, the RandomForestClassifier is removed
        clf1 = LogisticRegression()
        clf2 = RandomForestClassifier()
        eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)]
        eclf.set_params(rf=None)
        """
        super(OrdinalClassifier, self)._set_params('estimator', **params)
        return self

    def get_params(self, deep=True):
        return super(OrdinalClassifier, self)._get_params('estimator', deep=deep)


In [40]:
pipe = Pipeline([
    ('mapper', mapper),
    ('clf', OrdinalClassifier(clf))
])

# Fit Model on Full Sample

In [41]:
fitted = pipe.fit(X=revisions.copy(), y=revisions['wp10'])

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [44]:
revisions['pred'] = pipe.predict(X = revisions.copy())

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


In [46]:
rev_ct = pd.crosstab(index=revisions["wp10"], 
                     columns=revisions["pred"])
# rev_ct.index= ["wp10", "pred"]
rev_ct

pred,Stub,Start,C,B,GA,FA
wp10,,,,,,
Stub,4942,523,17,3,1,0
Start,1029,3819,502,98,25,3
C,164,1765,2835,299,301,121
B,111,1293,2102,1133,449,398
GA,131,258,1260,212,2294,1340
FA,114,39,276,236,701,3630


In [56]:
for cat in ("Stub", "Start", "C", "B", "GA", "FA"): 
    print(cat, np.mean((revisions["pred"] <= cat) == (revisions["wp10"] <= cat)))

Stub 0.935449050086
Start 0.860473723168
C 0.801535899334
B 0.882062669627
GA 0.90044411547
FA 1.0


In [ ]:
fitted.score(X=revisions.copy(), 

In [ ]:
with open("../models/model.pkl", "wb")  as f:
    dill.dump(fitted, f)

In [ ]:

def _parallel_fit_estimator2(estimator, X, y, cat):
    """Private function used to fit an estimator to a class within a job."""
    y_transformed = y > cat
    estimator.fit(X, y_transformed)
    return estimator


class OrdinalClassifier(_BaseComposition, ClassifierMixin, TransformerMixin):

    def __init__(self, estimator, n_jobs=None, proba_transform=None, left=True):
        self.estimator = estimator
        self.n_jobs = n_jobs
        self.proba_transform = proba_transform

    @property
    def named_estimators(self):
        return Bunch(**dict(self.estimator))

    def fit(self, X, y, categories='auto'):
        if not (isinstance(y, pd.Series) and hasattr(y, "cat")):
            raise ValueError("y must be pd.Series object with dtype Categorical")

        # this is hard-coded for categorical variables
        self.classes_ = y.cat.categories

        categories = self.classes_[1:]
    
        # order of estimators
        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
                delayed(_parallel_fit_estimator)(clone(self.estimator), X, y, cat)
                for cat in categories)

        self.named_estimators_ = Bunch(**dict())
        for k, e in zip(self.classes_[1:], self.estimators_):
            self.named_estimators_[k] = e
        return self

    def predict(self, X):
        out = np.argmax(self.predict_proba(X), axis=1)
        out = pd.Categorical.from_codes(out, categories=self.classes_, ordered=True)
        return out

    def _collect_log_probas(self, X):
        """Collect results from clf.predict calls. """
        if hasattr(clf, "predict_log_proba"):
            return [clf.predict_log_proba(X) for clf in self.estimators_]
        else:
            return [np.log(clf.predict_proba(X)) for clf in self.estimators_]

    def _predict_log_proba(self, X):
        """Predict log class probabilities for X"""
        out = np.empty((X.shape[0], len(self.classes_)))
        for i, logp in enumerate(self._collect_log_probas(X)):
            if i > 0:
                # add log conditional probability
                logp += out[:, (i, )]
            out[:, i:(i + 2)] = logp
        return out

    @property
    def predict_log_proba(self):
        """Compute probabilities of possible outcomes for samples in X.
        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        Returns
        ----------
        avg : array-like, shape = [n_samples, n_classes]
            Weighted average probability for each class per sample.
        """
        return self._predict_log_proba
    
    def predict_proba(self, X):
        return np.exp(self.predict_log_proba(X))

    def transform(self, X):
        if self.proba_transform:
            return self.predict_proba(X)
        else:
            return self.predict(X)
    
    def set_params(self, **params):
        """ Setting the parameters for the voting classifier
        Valid parameter keys can be listed with get_params().
        Parameters
        ----------
        **params : keyword arguments
            Specific parameters using e.g. set_params(parameter_name=new_value)
            In addition, to setting the parameters of the ``VotingClassifier``,
            the individual classifiers of the ``VotingClassifier`` can also be
            set or replaced by setting them to None.
        Examples
        --------
        # In this example, the RandomForestClassifier is removed
        clf1 = LogisticRegression()
        clf2 = RandomForestClassifier()
        eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2)]
        eclf.set_params(rf=None)
        """
        super(OrdinalClassifier, self)._set_params('estimator', **params)
        return self

    def get_params(self, deep=True):
        return super(OrdinalClassifier, self)._get_params('estimator', deep=deep)


# Evaluate Model Peformance

# TODO

1. Cross validate
2. Out of sample
3. Other